## ANALYZE
the data

This is a work in progress  

Vectorize the data in beer.review  
Diminish the importance of common words  
Compare ML algorithms to use the review data to predict beer.style  

In [2]:
# get modueles and get data
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

pd.set_option('max_colwidth', 300)
df = pd.read_csv('beer.csv', header=0)
df_copy = df  #save a copy of dataframe for reference. 
print('length',len(df))


df.head(3)

length 80818


,name,brewery,style,rating,review
0,Big Rock Ale,Big Rock Brewery,Scottish Ale,3.90,"smell  soft hop aroma with significant malt scents. this one smells very creamy. taste  and creamy it is. the traditional irish flavors come out at the tongue. this is creamy, not like a cream ale, but close. the malt is big, buttery, and very smooth. the hops are very unique. this is not a sh..."
1,Flip Ale,Dogfish Head Craft Brewery,Old Ale,4.08,on tap at dfh rehoboth... collab with eatily... cardamom and red wine must. golden orange. .no head. typical dfh yeast aroma. ..some spice and maybe a belgian influence. sweet spicy and somewhat fruity.. not much old ale characteristic. too light for that. its still tasty. the cardamom does add...
2,The Almond Marzen Project - Beer Camp #26,Sierra Nevada Brewing Co.,Märzen / Oktoberfest,3.78,"nice auburn impressions, tons of clarity, solid inch of off white head. aroma was a little bit sweet and nutty. taste gave a little more sweetness, stayed away from hops and bitterness, relatively light bodied. nothing almond came out of it that was obvious. kind of a fancied up oktoberfest ..."


In [3]:
#prep the data
df = df.drop(['name','brewery'], axis=1)
# drop all reviews with < 20 characters
df = df[df['review'].map(len) > 20]
print('length',len(df))

# remove uncommon styles (in EDA, I found 13 styles with fewer than 60 reviews)
#uncommon = [ 'American Dark Wheat Ale','Bière de Champagne / Bière Brut', 'Black & Tan', 
#            'Eisbock', 'Faro', 'Gueuze', 'Happoshu', 'Japanese Rice Lager', 'Kristalweizen',
#            'Kvass', 'Lambic - Unblended','Roggenbier', 'Sahti' ]   
labels = df.groupby(['style']).size() 
uncommon = labels[labels<150]
df = df.loc[~df['style'].isin(uncommon.index)]
print('length',len(df))

X = df['review'].values
y = df['style'].values
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=22)

print('total reviews:', format(len(df), ',d'))

length 49141
length 46861
total reviews: 46,861


In [4]:
print('X_train:',len(X_train), 'y_train:',len(y_train))
print('X_test:', len(X_test), 'y_test:', len(y_test))

X_train: 32802 y_train: 32802
X_test: 14059 y_test: 14059


In [5]:
print('number of styles excluded:',len(uncommon))
print(uncommon.index)

number of styles excluded: 30
Index(['American Dark Wheat Ale', 'American Double / Imperial Pilsner',
       'American Malt Liquor', 'Bière de Champagne / Bière Brut',
       'Black & Tan', 'Braggot', 'California Common / Steam Beer',
       'Chile Beer', 'Dortmunder / Export Lager', 'Eisbock',
       'English Pale Mild Ale', 'English Strong Ale', 'Euro Strong Lager',
       'Faro', 'Flanders Oud Bruin', 'Flanders Red Ale',
       'Foreign / Export Stout', 'Gueuze', 'Happoshu', 'Japanese Rice Lager',
       'Kristalweizen', 'Kvass', 'Lambic - Unblended', 'Low Alcohol Beer',
       'Rauchbier', 'Roggenbier', 'Sahti',
       'Scottish Gruit / Ancient Herbed Ale', 'Weizenbock', 'Wheatwine'],
      dtype='object', name='style')


In [6]:
beer_styles = df.groupby(['style']).size() 
print('number of styles included:',len(beer_styles))
print(beer_styles.index)

number of styles included: 74
Index(['Altbier', 'American Adjunct Lager', 'American Amber / Red Ale',
       'American Amber / Red Lager', 'American Barleywine',
       'American Black Ale', 'American Blonde Ale', 'American Brown Ale',
       'American Double / Imperial IPA', 'American Double / Imperial Stout',
       'American IPA', 'American Pale Ale (APA)', 'American Pale Lager',
       'American Pale Wheat Ale', 'American Porter', 'American Stout',
       'American Strong Ale', 'American Wild Ale', 'Baltic Porter',
       'Belgian Dark Ale', 'Belgian IPA', 'Belgian Pale Ale',
       'Belgian Strong Dark Ale', 'Belgian Strong Pale Ale',
       'Berliner Weissbier', 'Bière de Garde', 'Bock', 'Cream Ale',
       'Czech Pilsener', 'Doppelbock', 'Dubbel', 'Dunkelweizen',
       'English Barleywine', 'English Bitter', 'English Brown Ale',
       'English Dark Mild Ale', 'English India Pale Ale (IPA)',
       'English Pale Ale', 'English Porter', 'English Stout',
       'Euro Dark Lager',

In [7]:
# Change review to a string of words.  remove non-letters, make lower case, split into words.  
# Remove stopwords (common words.)  Join back together into a long string of words.  

def review_to_words(review):
    letters_only = re.sub('[^a-zA-Z]',' ', review)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))  
    good_words = [w for w in words if not w in stops]
    return(' '.join(good_words))

In [8]:
# clean up train reviews
num_reviews = len(X_train)
clean_train_reviews = []
for b in range(0, num_reviews):
    clean_train_reviews.append(review_to_words(X_train[b]))

In [9]:
#clean up test reviews
num_reviews = len(X_test)
clean_test_reviews = []
for b in range(0, num_reviews):
    clean_test_reviews.append(review_to_words(X_test[b]))

In [10]:
clean_train_reviews[0]

'one last pours gravity head brewed new holland brewing co restaurant pub chef cleetus friedman city provisions delicatessen named infamous marsha mallow nhb staff holiday party black head nice amount velvety lacing smoke chocolate malt huge malt smell takes sweet little sweet big chocolate malt espresso slight honey charred wood coats well want want stout thick little carbonation syrupy slight burn finish never glad tried nice example milk stout great mouthfeel overall way sweet took away complexity brew'

In [11]:
clean_test_reviews[2]

'scuttlebutt ipa tap brewforia golden honey color tiny bit haze carbonation ok lacing nose soft fruit seems hop forward tongue malts work make nice backbone finishes grapefruit dryness mf soft light mouthfeel little drying sessionable notes beer definitely dry hop forward traditional west coast ipa jump conclusion english ipa either somewhere middle good way'

In [12]:
# vectorize the train data, fit and transform into feature vectors
vectorizer = CountVectorizer(analyzer='word')
train_data_features = vectorizer.fit_transform(clean_train_reviews)
# arrays are easy to work with, so:
train_data_features = train_data_features.toarray()

In [13]:
vocab = vectorizer.get_feature_names()
print('first 120 words:', vocab[:120])

first 120 words: ['aa', 'aaa', 'aaaaand', 'aaaack', 'aaaah', 'aaaahs', 'aaaand', 'aaah', 'aaahh', 'aaaiiyeeee', 'aachen', 'aachener', 'aack', 'aah', 'aahh', 'aaj', 'aal', 'aalborg', 'aalmost', 'aals', 'aan', 'aand', 'aantal', 'aapa', 'aapect', 'aara', 'aarde', 'aargh', 'aaron', 'aarrgh', 'aas', 'aasher', 'aass', 'aat', 'ab', 'aba', 'aback', 'abacus', 'abadia', 'abale', 'abandon', 'abandoned', 'abandoning', 'abandons', 'abarleywine', 'abarwithnoname', 'abas', 'abashed', 'abate', 'abated', 'abates', 'abating', 'abavarian', 'abb', 'abbaye', 'abbazia', 'abbbey', 'abbey', 'abbeys', 'abbot', 'abbott', 'abbreviate', 'abbreviated', 'abbreviation', 'abbundantly', 'abby', 'abbyaye', 'abbywinters', 'abc', 'abcense', 'abd', 'abdij', 'abdijbier', 'abdomen', 'abducting', 'abduction', 'abe', 'abeba', 'abeille', 'abeit', 'abel', 'aberdeen', 'aberlour', 'aberrartion', 'aberration', 'aberrations', 'abesentia', 'abet', 'abetted', 'abetter', 'abetters', 'abetting', 'abettor', 'abeyance', 'abf', 'abfod', '

In [14]:
dist = np.sum(train_data_features, axis=1)
for tag, count in zip(vocab, dist):
    print(count,tag)[:40]

78 aa


TypeError: 'NoneType' object is not subscriptable

In [64]:
text_clf= Pipeline([('vect', CountVectorizer(min_df=7)),
                   ('tfidf', TfidfTransformer()),
                   ('clf',MultinomialNB()) ])
# first pass, .1195
# after word cleaning, .2076
text_clf = text_clf.fit(clean_train_reviews, y_train)
predicted = text_clf.predict(clean_test_reviews)
np.mean(predicted == y_test)

0.20755388007681913

0.11948877468217474

In [ ]:
X_train2 = X_train[:200]
X_train2 = y_train[:200]

print(X_train2.shape,X_train2.shape)

In [65]:
# RANDOM FOREST 

clean_train_reviews2 = clean_train_reviews[:5000]
y_train2 = y_train[:5000]
clean_test_reviews2 = clean_test_reviews[10000:10500]
y_test2 = y_test[10000:10500]

from sklearn.ensemble import RandomForestClassifier as RFC
text_clf= Pipeline([('vect', CountVectorizer(min_df=5)),
                   ('tfidf', TfidfTransformer()),
                   ('forest',RFC(n_estimators=300)) ])
text_clf = text_clf.fit(clean_train_reviews2, y_train2)
predicted = text_clf.predict(clean_test_reviews2)
np.mean(predicted == y_test2)

0.44

In [66]:
# LOGISTIC REGRESSION
from sklearn.model_selection import cross_val_score

# vectorize the train data, fit and transform into feature vectors
text_clf= Pipeline([('vect', CountVectorizer(min_df=5)),
                   ('tfidf', TfidfTransformer()),
                   ('clf',LogisticRegression()) ])
text_clf = text_clf.fit(clean_train_reviews, y_train)
predicted = text_clf.predict(clean_test_reviews)
np.mean(predicted == y_test)

#scores = cross_val_score(LogisticRegression(), train_data_features, y_train, cv=5)
#print('mean cross-val accuracy: {:.2f}'.format(np.mean(scores)))

0.57500533466107118

In [ ]:
#this is quite slow.  don't run it until I can improve speed (min_df=5 is a start)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001,0.01, 0.1, 1,10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(clean_train_reviews, y_train)
print('best cross-val score: {:.2f}'.format(grid.gest_score_))
print('best params:', grid.best_params_)

In [ ]:
num_reviews = len(X_test)
clean_test_reviews = []
for b in range(0, num_reviews):
    clean_test_reviews.append(review_to_words(X_test[b]))
    
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
result = forest.predict(test_data_features)
output = pd.DataFrame(data = {'id':test['id'], 'style':result})
output_to_csv('BagOfWords_model.csv', index=False, quoting=3)

In [118]:
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [119]:
X2 = [df['review'][0]] 

vectorizer = CountVectorizer()
vectorizer.fit(X)
vector = vectorizer.transform(X2)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

(1, 384)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 0 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 2 0 0 0 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 2 0 0 0 2 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 0 2 7 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0 1 0 0 0 1 0 0 0 0 4 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [120]:
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [121]:
vector = vectorizer.transform(X2)
print(vector.toarray())

[[0 0 0 0 1 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 2 0 0 0 0 0 1 0 1 0 0 0 0 0
  0 0 0 0 0 2 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
  0 0 0 2 0 0 0 2 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
  0 0 2 7 0 0 0 0 0 6 0 0 0 0 2 0 0 0 0 0 1 0 0 0 1 0 0 0 0 4 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(X)
# summarize
print(len(str(vectorizer.vocabulary_)))
print(str(vectorizer.vocabulary_)[:600])

5426
{'nice': 220, 'auburn': 31, 'impressions': 163, 'tons': 350, 'of': 232, 'clarity': 72, 'solid': 308, 'inch': 165, 'off': 233, 'white': 372, 'head': 144, 'aroma': 28, 'was': 364, 'little': 192, 'bit': 45, 'sweet': 327, 'and': 21, 'nutty': 229, 'taste': 331, 'gave': 125, 'more': 213, 'sweetness': 328, 'stayed': 316, 'away': 33, 'from': 121, 'hops': 153, 'bitterness': 47, 'relatively': 278, 'light': 188, 'bodied': 50, 'nothing': 226, 'almond': 11, 'came': 61, 'out': 240, 'it': 171, 'that': 335, 'obvious': 230, 'kind': 177, 'fancied': 112, 'up': 358, 'oktoberfest': 234, 'while': 371, 'good': 132, 


In [113]:
print(vectorizer.idf_)

[ 6.20350706  7.90825515  7.90825515 ...,  7.50279005  7.90825515
  7.90825515]


In [98]:
# encode document
vector = vectorizer.transform([X[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 384)
[[ 0.01357063  0.01357063  0.0407119   0.01357063  0.02714126  0.01357063
   0.01357063  0.01357063  0.01357063  0.01357063  0.02714126  0.05428253
   0.13570632  0.01357063  0.01357063  0.01357063  0.08142379  0.01357063
   0.01357063  0.01357063  0.02714126  0.36640707  0.01357063  0.01357063
   0.02714126  0.01357063  0.01357063  0.0407119   0.01357063  0.01357063
   0.14927696  0.01357063  0.01357063  0.01357063  0.02714126  0.01357063
   0.01357063  0.01357063  0.01357063  0.05428253  0.13570632  0.01357063
   0.01357063  0.01357063  0.02714126  0.01357063  0.01357063  0.01357063
   0.01357063  0.01357063  0.02714126  0.0407119   0.01357063  0.01357063
   0.01357063  0.0407119   0.02714126  0.0407119   0.01357063  0.10856506
   0.02714126  0.02714126  0.02714126  0.01357063  0.01357063  0.01357063
   0.0407119   0.02714126  0.01357063  0.01357063  0.01357063  0.01357063
   0.02714126  0.06785316  0.0407119   0.01357063  0.0407119   0.02714126
   0.0407119   0.01357063  0.

In [104]:
# vectorize with a list of reviews

from sklearn.feature_extraction.text import TfidfVectorizer
X = df['review'][0:2000]

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(X)
# summarize
print(str(vectorizer.vocabulary_)[:1000])  # print a slice because this is very long
print(len(str(vectorizer.vocabulary_)))

{'smell': 11165, 'soft': 11263, 'hop': 6090, 'aroma': 918, 'with': 13621, 'significant': 10951, 'malt': 7438, 'scents': 10579, 'this': 12373, 'one': 8499, 'smells': 11171, 'very': 13165, 'creamy': 3179, 'taste': 12183, 'and': 760, 'it': 6575, 'is': 6562, 'the': 12317, 'traditional': 12594, 'irish': 6546, 'flavors': 4964, 'come': 2790, 'out': 8606, 'at': 1005, 'tongue': 12522, 'not': 8299, 'like': 7146, 'cream': 3171, 'ale': 629, 'but': 2030, 'close': 2653, 'big': 1452, 'buttery': 2039, 'smooth': 11194, 'hops': 6116, 'are': 896, 'unique': 12939, 'sharp': 10812, 'flavor': 4957, 'an': 750, 'easy': 4133, 'saturated': 10524, 'well': 13435, 'mixed': 7865, 'blend': 1560, 'that': 12312, 'plays': 9177, 'complimenting': 2866, 'second': 10666, 'fiddle': 4818, 'to': 12482, 'base': 1222, 'no': 8248, 'sweetness': 12034, 'finish': 4878, 'nutty': 8357, 'changes': 2388, 'personalities': 9001, 'end': 4289, 'mouthfeel': 8004, 'lightly': 7135, 'carbonated': 2203, 'exceptionally': 4515, 'drinkability': 400

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer

X = [df['review'][2]]  # start with a long review
X2 = [df['review'][0]] # X2 is shorter

# create the transform
vectorizer = HashingVectorizer(n_features=50)
# encode document
vector = vectorizer.transform(X)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 50)
[[ 0.07887912  0.01314652 -0.0657326   0.02629304 -0.0657326  -0.17090475
  -0.01314652  0.         -0.28922343  0.01314652  0.10517215  0.01314652
   0.28922343 -0.14461171 -0.01314652  0.01314652 -0.02629304 -0.10517215
   0.21034431  0.          0.07887912  0.14461171 -0.07887912  0.02629304
  -0.21034431  0.07887912 -0.14461171  0.14461171  0.01314652  0.
   0.02629304 -0.03943956  0.24978387  0.14461171  0.07887912  0.15775823
   0.07887912 -0.02629304  0.23663735  0.13146519 -0.22349083  0.19719779
   0.         -0.03943956  0.07887912 -0.36810254  0.36810254 -0.07887912
  -0.03943956  0.01314652]]


In [31]:
counts = df.groupby(['style']).size()
styles = counts.index

In [32]:
styles

Index(['Altbier', 'American Adjunct Lager', 'American Amber / Red Ale',
       'American Amber / Red Lager', 'American Barleywine',
       'American Black Ale', 'American Blonde Ale', 'American Brown Ale',
       'American Dark Wheat Ale', 'American Double / Imperial IPA',
       ...
       'Scotch Ale / Wee Heavy', 'Scottish Ale',
       'Scottish Gruit / Ancient Herbed Ale', 'Smoked Beer', 'Tripel',
       'Vienna Lager', 'Weizenbock', 'Wheatwine', 'Winter Warmer', 'Witbier'],
      dtype='object', name='style', length=104)

In [51]:
# vectorize with a list of reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#  Create X for predictor variables, y for target
X = df['review'][0:2000].values
y = df['style'][0:2000].values
test_X = df['review'][3000:4000].values
test_y = df['style'][3000:4000].values

# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X)

# Naive Bayes classifier
clf = MultinomialNB().fit(X_train, y)

In [52]:
predicted = clf.predict(test_X)
np.mean(predicted == test_y)

ValueError: could not convert string to float: "   poured from a 650ml bottle,7% abv, 80 ibu's fresh hopped with cascade and centennial with nugget for bittering. l: pours a dark red amber with just a skim of head with minor lacing. s: not much of a nose for me ,some hops and mild spice. t: really smooth well balance red ipa, looks like they used 5 malts and it works well with the fresh hops nice on the bitterness. f: a light carbonation nice mouth feel no alcohol taste at all. o: a really nice fresh hopped red ipa.   "

In [ ]:
# could use a pipeline to do the same:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),                      
                     ('clf', MultinomialNB()),])
text_clf = text_clf.fit(X,y)

### start over here

In [60]:
df = df_copy

In [66]:
#prep the data
df = df_copy
df = df.drop(['name','brewery'], axis=1)
# drop all reviews with < 15 characters
df = df[df['review'].map(len) > 14]

In [68]:
print(len(df))

49291


In [69]:
t_id = 0
text = df.loc[t_id, 'review']

In [72]:
from gensim.summarization import summarize, keywords

word_scores = keywords(text, words=5, scores=True, split=True, lemmatize=True)

ModuleNotFoundError: No module named 'gensim'